In [26]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow narwhals 

In [27]:
import pandas as pd
import polars as pl

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True

In [28]:
from typing import Any
import narwhals as nw
from datetime import datetime

def q15(
    lineitem_ds_raw: Any,
    supplier_ds_raw: Any,
) -> Any:

    lineitem_ds = nw.from_native(lineitem_ds_raw)
    supplier_ds = nw.from_native(supplier_ds_raw)
    
    var1 = datetime(1996, 1, 1)
    var2 = datetime(1996, 4, 1)

    revenue = (
        lineitem_ds.filter(nw.col("l_shipdate").is_between(var1, var2, closed="left"))
        .with_columns(
            (nw.col("l_extendedprice") * (1 - nw.col("l_discount")))
            .alias("total_revenue")
        )
        .group_by("l_suppkey")
        .agg(nw.sum("total_revenue"))
        .select(nw.col("l_suppkey").alias("supplier_no"), nw.col("total_revenue"))
    )
    
    result = (
        supplier_ds.join(revenue, left_on="s_suppkey", right_on="supplier_no")
        .filter(nw.col("total_revenue") == nw.col("total_revenue").max())
        .with_columns(nw.col("total_revenue").round(2))
        .select("s_suppkey", "s_name", "s_address", "s_phone", "total_revenue")
        .sort("s_suppkey")
    )

    return nw.to_native(result)

In [29]:
dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"
lineitem = dir_ + 'lineitem.parquet'
supplier = dir_ + 'supplier.parquet'

In [30]:
IO_FUNCS = {
    'pandas': lambda x: pd.read_parquet(x, engine='pyarrow'),
    'pandas[pyarrow]': lambda x: pd.read_parquet(x, engine='pyarrow', dtype_backend='pyarrow'),
    'polars[eager]': lambda x: pl.read_parquet(x),
    'polars[lazy]': lambda x: pl.scan_parquet(x),
}

In [31]:
results = {}

## pandas via Narwhals

In [32]:
tool = 'pandas'
fn = IO_FUNCS[tool]
timings = %timeit -o q15(fn(lineitem), fn(supplier))
results[tool] = timings.all_runs

5.14 s ± 147 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## pandas, pyarrow dtypes, via Narwhals

In [33]:
tool = 'pandas[pyarrow]'
fn = IO_FUNCS[tool]
timings = %timeit -o q15(fn(lineitem), fn(supplier))
results[tool] = timings.all_runs

3.52 s ± 74.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Polars read_parquet

In [34]:
tool = 'polars[eager]'
fn = IO_FUNCS[tool]
timings = %timeit -o q15(fn(lineitem), fn(supplier))
results[tool] = timings.all_runs

2.7 s ± 673 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Polars scan_parquet

In [35]:
tool = 'polars[lazy]'
fn = IO_FUNCS[tool]
timings = %timeit -o q15(fn(lineitem), fn(supplier)).collect()
results[tool] = timings.all_runs

446 ms ± 7.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Save

In [36]:
import json
with open('results.json', 'w') as fd:
    json.dump(results, fd)
